# 🌟 Phase 1, Week 3: Entanglement & Bell States - The Spooky Connection! 🌟

Prepare to have your mind stretched! This week, we're exploring one of the most profound and counter-intuitive phenomena in quantum mechanics: **Entanglement**. Often called "spooky action at a distance" by Einstein, entanglement describes a connection between quantum particles so deep that they share the same fate, no matter how far apart they are!

Our primary goals for this week's coding project are to:
* **Code a Bell state simulator**
* **Simulate correlation measurements** for these entangled states

Through these simulations, we'll gain a practical understanding of:
* **Entanglement** itself
* The famous **EPR (Einstein-Podolsky-Rosen) paradox**, which highlights the non-local nature of entanglement

---

## 🚀 1. Building Bell States: The Foundation of Entanglement

Bell states are a set of four maximally entangled two-qubit states. They are the simplest examples of entanglement and are foundational in quantum information science. They look like this:

* $|\Phi^+\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$
* $|\Phi^-\rangle = \frac{1}{\sqrt{2}}(|00\rangle - |11\rangle)$
* $|\Psi^+\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)$
* $|\Psi^-\rangle = \frac{1}{\sqrt{2}}(|01\rangle - |10\rangle)$

To create these in QuTiP, we'll need to define a two-qubit system. This involves combining single-qubit basis states using the `tensor` product. A common way to create a Bell state (e.g., $|\Phi^+\rangle$) is to start with $|00\rangle$, apply a Hadamard gate to the first qubit, and then apply a CNOT (Controlled-NOT) gate where the first qubit controls the second.

---

## 📊 2. Measuring Correlations: The Proof of Entanglement

The magic of Bell states truly shines when you measure them. If you measure one qubit of an entangled pair, you instantly know the state of the other, even if it's light-years away! This perfect correlation is a hallmark of entanglement.

We'll simulate sequential measurements on both qubits of a Bell state to observe these correlations. For example, if you measure the first qubit of $|\Phi^+\rangle$ and find it in $|0\rangle$, you are guaranteed to find the second qubit also in $|0\rangle$.

---

## 📈 3. Simulating Measurements on Entangled Qubits (Similar to Week 2, but now for two qubits!)

Just like last week, QuTiP doesn't directly "collapse" the state for you after a measurement. Instead, we'll:
1.  Define the projection operators for two-qubit states (e.g., $|00\rangle\langle 00|$, $|01\rangle\langle 01|$, etc.).
2.  Calculate the probabilities of measuring each of the four possible two-qubit outcomes ($|00\rangle$, $|01\rangle$, $|10\rangle$, $|11\rangle$).
3.  Use random sampling based on these probabilities to simulate multiple measurements and observe the correlations.

This will allow us to demonstrate the "spooky action" in action!

In [96]:
import numpy as np
from qutip import basis, tensor
from qutip.qip.operations import hadamard_transform, cnot

# --- 1. Building Bell States: The Foundation of Entanglement ---

# Define single qubit basis states
ket_0 = basis(2, 0) # |0>
ket_1 = basis(2, 1) # |1>

# Define two-qubit basis states using tensor products
# |00>
ket_00 = tensor(ket_0, ket_0)
# |01>
ket_01 = tensor(ket_0, ket_1)
# |10>
ket_10 = tensor(ket_1, ket_0)
# |11>
ket_11 = tensor(ket_1, ket_1)

print("Two-qubit basis states:")
# print(f"|00>:\n{ket_00}")
# print(f"|01>:\n{ket_01}")
# print(f"|10>:\n{ket_10}")
# print(f"|11>:\n{ket_11}\n")

# Let's create the Bell state |Φ+⟩ = (1/√2)(|00⟩ + |11⟩)
# Initial state is |00>
psi_bell = ket_00

# Apply Hadamard gate to the first qubit H ⊗ I
H = hadamard_transform()  # Hadamard gate on the first qubit
# Identity operator for a qubit
I = basis(2, 0) * basis(2, 0).dag() + basis(2, 1) * basis(2, 1).dag()

H_tensor_I = tensor(H, I)

psi_bell = H_tensor_I * psi_bell

print("State after Hadamard on first qubit (Hadamard ⊗ I) |00>:\n", psi_bell)
# Apply CNOT gate (first qubit control, second qubit target)
# In QuTiP, cnot() by default acts on 2 qubits with control=0, target=1
psi_bell = cnot() * (psi_bell)
print("Generated Bell state |Φ+⟩ (after CNOT):\n", psi_bell)

# --- 2. Simulating Measurements on Entangled Qubits ---

# Define projection operators for each of the four possible two-qubit outcomes
P_00 = ket_00 * ket_00.dag()  # |00⟩⟨00|
P_01 = ket_01 * ket_01.dag()  # |01⟩⟨01|
P_10 = ket_10 * ket_10.dag()  # |10⟩⟨10|
P_11 = ket_11 * ket_11.dag()  # |11⟩⟨11|

# Calculate probabilities of measuring each outcome
prob_00 = (psi_bell.dag() * P_00 * psi_bell).real
prob_01 = (psi_bell.dag() * P_01 * psi_bell).real
prob_10 = (psi_bell.dag() * P_10 * psi_bell).real
prob_11 = (psi_bell.dag() * P_11 * psi_bell).real

print(f"\nProbabilities for Bell State |Φ+⟩:")
print(f"  P(|00⟩): {prob_00:.4f}")
print(f"  P(|01⟩): {prob_01:.4f}")
print(f"  P(|10⟩): {prob_10:.4f}")
print(f"  P(|11⟩): {prob_11:.4f}")
print(f"  Sum of probabilities: {prob_00 + prob_01 + prob_10 + prob_11:.4f} (should be 1.0)\n")

# --- 3. Simulate Multiple Measurements to Observe Correlations ---
num_measurements = 1000
measurement_results = [] # Store tuple (q0_result, q1_result)

# Possible outcomes for np.random.choice (indices for 00, 01, 10, 11)
outcomes_idx = [0, 1, 2, 3] # Corresponds to ket_00, ket_01, ket_10, ket_11
probabilities = [prob_00, prob_01, prob_10, prob_11]

for _ in range(num_measurements):
    chosen_idx = np.random.choice(outcomes_idx, p=probabilities)

    if chosen_idx == 0: # |00>
        measurement_results.append((0, 0))
    elif chosen_idx == 1: # |01>
        measurement_results.append((0, 1))
    elif chosen_idx == 2: # |10>
        measurement_results.append((1, 0))
    elif chosen_idx == 3: # |11>
        measurement_results.append((1, 1))

# Analyze the results
count_00 = measurement_results.count((0, 0))
count_01 = measurement_results.count((0, 1))
count_10 = measurement_results.count((1, 0))
count_11 = measurement_results.count((1, 1))

print(f"--- Simulating {num_measurements} measurements on |Φ+⟩ ---")
print(f"Observed |00⟩: {count_00} times ({count_00/num_measurements:.3f})")
print(f"Observed |01⟩: {count_01} times ({count_01/num_measurements:.3f})")
print(f"Observed |10⟩: {count_10} times ({count_10/num_measurements:.3f})")
print(f"Observed |11⟩: {count_11} times ({count_11/num_measurements:.3f})")

print("\nNotice the strong correlations! For |Φ+⟩, we expect only |00⟩ and |11⟩ outcomes.")
print("This demonstrates entanglement: if the first qubit is 0, the second MUST be 0, and vice-versa.")



Two-qubit basis states:
State after Hadamard on first qubit (Hadamard ⊗ I) |00>:
 Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.70710678]
 [0.        ]
 [0.70710678]
 [0.        ]]
Generated Bell state |Φ+⟩ (after CNOT):
 Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.70710678]
 [0.        ]
 [0.        ]
 [0.70710678]]

Probabilities for Bell State |Φ+⟩:
  P(|00⟩): 0.5000
  P(|01⟩): 0.0000
  P(|10⟩): 0.0000
  P(|11⟩): 0.5000
  Sum of probabilities: 1.0000 (should be 1.0)

--- Simulating 1000 measurements on |Φ+⟩ ---
Observed |00⟩: 497 times (0.497)
Observed |01⟩: 0 times (0.000)
Observed |10⟩: 0 times (0.000)
Observed |11⟩: 503 times (0.503)

Notice the strong correlations! For |Φ+⟩, we expect only |00⟩ and |11⟩ outcomes.
This demonstrates entanglement: if the first qubit is 0, the second MUST be 0, and vice-versa.
